In [ ]:
'''
Calculate RMSE when simply use optical flow to measure vehicle's velocity:
    by identifying the valid velocity (speed) when length of track >50
    in raw data, round means every  4 seconds' refresh, e.g. in video 1 , there are 192 rounds
    item 0- xx in this round refers one corner
    
    result:
    video 1: 79.4
    video 2: 21.9
    video 3: 26.6
    video 4: 25.7

'''

In [14]:
import os 
import pandas as pd
import numpy as np

root = os.path.join(os.environ['HOME'], './')
data_dir = os.path.join(root, './result/track_data')
# data_with_label = pd.read_csv(os.path.join(data_dir, 'tracks_with_label_video3.csv'), index_col= None)
data_raw = pd.read_csv(os.path.join(data_dir, 'tracks_video4.csv'), index_col= None)
coner_ids = data_with_label['id'].unique()
len(coner_ids)

113

In [15]:
len(data_raw['round'].unique())

152

In [16]:
'''
2 filter track data
imput: track_xxx.csv 
select tracks which valid data is more than 80, e.g. from 50 tracks crated by optical flow
assign new id number

output: tracks_df
1st round filter: those tracks is shorter than 80 instances, by ids created


round: every 120 frames (4 secs, created by trackData_generation_optical_flow.py
id: id_num,  every set of track, e.g. 64 tracks before 2 times of fitering
n: number selected tracks in every round, e.g 10 in round 3 (example in old_new1.csv)

'''
import numpy as np
import pandas as pd
from tqdm import tqdm
import math
import warnings
warnings.filterwarnings("ignore")

data = data_raw
def clip_track(df, y_min, y_max): 
    df = df[df['y_old'] >= y_min]
    df = df[df['y_old'] <= y_max]
    return df

tracks_final = pd.DataFrame()
id_num = 0
tracks = []
square_error = 0
for rnd in tqdm(range(len(np.unique(data['round'])))):  ## travese by 'round', every rounds is 120 frames, 4 secs
    df =data[data['round']== rnd]
    df['pixels_per_frame'] = np.sqrt((df.new_x-df.old_x)**2 + (df.new_y-df.old_y)**2) ## add a new column of distance
    df= df[df['pixels_per_frame']>1]
    idx= df.index
    '''
    find the valuable trackings
    in the list with value of [x,y,pixels_per_frame] for each tracking
    '''
#     n  = 0
    pixel_speed_list_by_all_track = []
    pixel_speed_list_by_valuable_track = [] # valuable means len(this_track)> 50, regards as ground truth
#     round_speed = 0
    
    for idx_prev in idx:  ## those tracks in this round
        this_track = []
        
        if idx_prev not in df.index: continue ## skip when this row is counted and removed
        for i in idx:
            if i not in df.index: continue ## skip when this row is counted and removed by pixels_per_frame<=1
            if df.loc[idx_prev]['new_x'] == df.loc[i]['old_x']:  ## count the continueus tracked coner
                this_track.append([df.loc[i]['old_x'], df.loc[i]['old_y'],\
                                   df.loc[i]['new_x'], df.loc[i]['new_y'],df.loc[i]['pixels_per_frame'],id_num])
                df.drop(index = idx_prev, axis = 0, inplace = True) ## remove the row of found point
                idx_prev = i  ## move to the next point(corner)
                
        id_num+=1
        
        if len(this_track) > 50:  ##  if len(this_track)> 10 :  ## ids with instance is longer than xx (e.g.50)
            speed_this_track = np.mean(np.array(this_track)[:,4])
            if len(this_track)>200:  ## clip 400-600
                pixel_speed_list_by_valuable_track.append(speed_this_track)
            else:
                pixel_speed_list_by_all_track.append(speed_this_track)
        if len(pixel_speed_list_by_valuable_track) == 0:
            pixel_speed_list_by_valuable_track =[0]
                
    squre_error_this_round = np.power(np.mean(pixel_speed_list_by_all_track)-np.mean(pixel_speed_list_by_valuable_track),2)

    if not math.isnan(squre_error_this_round):
#         print('This rounds RMSE with pure optical flow method is:',squre_error_this_round)
        square_error+=squre_error_this_round

print('Total RMSE with pure optical flow method is in this video:', np.sqrt(square_error))

100%|█████████████████████████████████████████| 152/152 [00:35<00:00,  4.32it/s]

Total RMSE with pure optical flow method is in this video: 25.67122037044866
